<a href="https://colab.research.google.com/github/mdias23i/DE-DataProcessing/blob/main/spark_streaming/examples/example_2_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usecase 2
- Reading data from "rate"
- Aggregating data by window time
- Checking results from query in memory

# Setting up PySpark

In [ ]:
%pip install pyspark

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()

# Write output in memory

In [2]:
import pyspark.sql.functions as F

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream in memory
query = (agg.writeStream
.format('memory')
.queryName('my_query')
.outputMode('complete')
.start()
)

In [38]:
spark.sql("select * from my_query order by window desc").show(10,False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|{2024-11-23 15:49:25, 2024-11-23 15:49:30}|9    |
|{2024-11-23 15:49:20, 2024-11-23 15:49:25}|50   |
|{2024-11-23 15:49:15, 2024-11-23 15:49:20}|50   |
|{2024-11-23 15:49:10, 2024-11-23 15:49:15}|50   |
|{2024-11-23 15:49:05, 2024-11-23 15:49:10}|50   |
|{2024-11-23 15:49:00, 2024-11-23 15:49:05}|50   |
|{2024-11-23 15:48:55, 2024-11-23 15:49:00}|50   |
|{2024-11-23 15:48:50, 2024-11-23 15:48:55}|50   |
|{2024-11-23 15:48:45, 2024-11-23 15:48:50}|50   |
|{2024-11-23 15:48:40, 2024-11-23 15:48:45}|50   |
+------------------------------------------+-----+
only showing top 10 rows



In [39]:
query.stop()

# Write output as parquet


In [40]:
!rm -rf content/output

In [41]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame

def save_parquet(df, batch_id):
  (df
   .withColumn("batch_id",F.lit(batch_id))
   .withColumn("load_time",F.current_timestamp())
   .write.mode("append")
   .parquet("content/output/rate_parquet")
  )

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withWatermark("timestamp", "5 seconds").withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream as parquet with foreachBatch
query = (agg.writeStream
.option('checkpointLocation', 'content/output/checkpoint')
.trigger(processingTime='20 seconds')
.outputMode('append')
.foreachBatch(save_parquet)
.start()
)


In [47]:
result = spark.read.format("parquet").load("content/output/rate_parquet/")
result.sort(F.asc("window")).show(100, False)

+------------------------------------------+-----+--------+--------------------------+
|window                                    |count|batch_id|load_time                 |
+------------------------------------------+-----+--------+--------------------------+
|{2024-11-23 15:54:20, 2024-11-23 15:54:25}|40   |2       |2024-11-23 15:55:00.269719|
|{2024-11-23 15:54:25, 2024-11-23 15:54:30}|50   |2       |2024-11-23 15:55:00.269719|
|{2024-11-23 15:54:30, 2024-11-23 15:54:35}|50   |3       |2024-11-23 15:55:20.194862|
|{2024-11-23 15:54:35, 2024-11-23 15:54:40}|50   |3       |2024-11-23 15:55:20.194862|
|{2024-11-23 15:54:40, 2024-11-23 15:54:45}|50   |3       |2024-11-23 15:55:20.194862|
|{2024-11-23 15:54:45, 2024-11-23 15:54:50}|50   |3       |2024-11-23 15:55:20.194862|
|{2024-11-23 15:54:50, 2024-11-23 15:54:55}|50   |4       |2024-11-23 15:55:40.27568 |
|{2024-11-23 15:54:55, 2024-11-23 15:55:00}|50   |4       |2024-11-23 15:55:40.27568 |
|{2024-11-23 15:55:00, 2024-11-23 15:55:05}

In [49]:
query.stop()

# Enrich data with faker

In [50]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.2 MB/s eta 0:00:00


In [51]:
!rm -rf content/output/events

In [56]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker

def insert_into_table(df, batch_id):
  fake = Faker()
  new_columns = {
      'name': F.lit(fake.name()),
      'address': F.lit(fake.address()),
      'email': F.lit(fake.email()),
      'dob': F.lit(fake.date_of_birth()),
      'phone': F.lit(fake.phone_number())
  }
  df = df.withColumns(new_columns)
  df.write.mode("append").format("parquet").save("content/output/events")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_into_table)
.start()
)

In [54]:
query.stop()


In [57]:
spark.read.parquet("content/output/events").show(100, False)

+-----------------------+-----+-------------------+----------------------------------------------------------+---------------------------+----------+---------------------+
|timestamp              |value|name               |address                                                   |email                      |dob       |phone                |
+-----------------------+-----+-------------------+----------------------------------------------------------+---------------------------+----------+---------------------+
|2024-11-23 16:02:14.308|33   |Jenna Gilbert      |119 Bradshaw Dam\nSouth Sharonberg, IA 43628              |jeremybrown@example.org    |1936-08-08|001-992-876-1588x3496|
|2024-11-23 16:02:15.308|34   |Jenna Gilbert      |119 Bradshaw Dam\nSouth Sharonberg, IA 43628              |jeremybrown@example.org    |1936-08-08|001-992-876-1588x3496|
|2024-11-23 16:02:23.308|42   |Mr. Robert Ortiz MD|4193 Christian Views Apt. 507\nEdwardchester, GA 70126    |wsellers@example.org       |19

In [58]:
query.stop()